In [1]:
from PandaStation import (
    PandaStation, FindResource, AddPackagePaths, AddRgbdSensors, draw_points, draw_open3d_point_cloud, 
    create_open3d_point_cloud, AddShape)
#from PandaGrasping import *


# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import open3d as o3d
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    MakeRenderEngineVtk, DepthRenderCamera, RenderCameraCore, CameraInfo, ClippingRange,  DepthImageToPointCloud,
    BaseField, RenderEngineVtkParams, ConnectMeshcatVisualizer, DepthRange, RgbdSensor, MeshcatPointCloudVisualizer,
    LoadModelDirectives, ProcessModelDirectives, GeometrySet
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

import matplotlib.pyplot as plt

In [2]:
def grasp_candidate_cost(X_G, plant_context, cloud, plant, scene_graph, scene_graph_context):
    
    body = plant.GetBodyByName("panda_hand")

    X_GW = X_G.inverse()
    pts = np.asarray(cloud.points).T
    p_GC = X_GW.multiply(pts)

    # Crop to a region inside of the finger box.
    crop_min = [-0.009, -0.035, 0.06]
    crop_max = [0.009, 0.035, 0.115]
    indices = np.all((crop_min[0] <= p_GC[0,:], p_GC[0,:] <= crop_max[0],
                      crop_min[1] <= p_GC[1,:], p_GC[1,:] <= crop_max[1],
                      crop_min[2] <= p_GC[2,:], p_GC[2,:] <= crop_max[2]), 
                     axis=0)


    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    collision_pairs = query_object.ComputePointPairPenetration()
    
    # check collisions with objects in the world 
    if query_object.HasCollisions():
        #print('collision with world')
        return np.inf

    # Check collisions between the gripper and the point cloud
    margin = 0.0  # must be smaller than the margin used in the point cloud preprocessing.
    for pt in cloud.points:
        distances = query_object.ComputeSignedDistanceToPoint(pt, threshold=margin)
        if distances:
            #print('collision with point cloud')
            return np.inf

    n_GC = X_GW.rotation().multiply(np.asarray(cloud.normals)[indices,:].T)

    # Penalize deviation of the gripper from vertical.
    # weight * -dot([0, 0, -1], R_G * [0, 1, 0]) = weight * R_G[2,1]
    cost = 20.0*X_G.rotation().matrix()[2, 1]

    # Reward sum |dot product of normals with gripper x|^2
    cost -= np.sum(n_GC[0,:]**2)

    return cost

def process_bin_point_cloud(diagram, context, cameras, bin_name):

    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    # Compute crop box.
    bin_instance = plant.GetModelInstanceByName(bin_name)
    bin_body = plant.GetBodyByName("bin_base", bin_instance)
    X_B = plant.EvalBodyPoseInWorld(plant_context, bin_body)
    margin = 0.001  # only because simulation is perfect!
    a = X_B.multiply([-.22+0.025+margin, -.29+0.025+margin, 0.015+margin])
    b = X_B.multiply([.22-0.1-margin, .29-0.025-margin, 2.0])
    crop_min = np.minimum(a,b)
    crop_max = np.maximum(a,b)

    # Evaluate the camera output ports to get the images.
    merged_pcd = o3d.geometry.PointCloud()
    for c in cameras:
        point_cloud = diagram.GetOutputPort(f"{c}_point_cloud").Eval(context)
        pcd = create_open3d_point_cloud(point_cloud)

        # Crop to region of interest.
        pcd = pcd.crop(
            o3d.geometry.AxisAlignedBoundingBox(min_bound=crop_min,
                                                max_bound=crop_max))    

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.1, max_nn=30))

        camera = plant.GetModelInstanceByName(c)
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd.orient_normals_towards_camera_location(X_C.translation())
        
        # Merge point clouds.
        merged_pcd += pcd

    # Voxelize down-sample.  (Note that the normals still look reasonable)
    return merged_pcd.voxel_down_sample(voxel_size=0.005)



def generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, panda, rng, meshcat=None, avoid_names = None):
    """
    Picks a random point in the cloud, and aligns the robot finger with the normal of that pixel. 
    The rotation around the normal axis is drawn from a uniform distribution over [min_roll, max_roll].
    """
    body = plant.GetBodyByName("panda_hand")

    index = rng.integers(0,len(cloud.points)-1)

    # Use S for sample point/frame.
    p_WS = np.asarray(cloud.points[index])
    n_WS = np.asarray(cloud.normals[index])

    if meshcat:
        vertices = np.empty((3,2))
        vertices[:, 0] = p_WS
        vertices[:, 1] = p_WS + 0.05*n_WS
        meshcat.set_object(g.LineSegments(g.PointsGeometry(vertices),
                            g.MeshBasicMaterial(color=0xff0000)))
  

    if not np.isclose(np.linalg.norm(n_WS), 1.0):
        return np.inf, None

    Gy = n_WS # gripper y axis aligns with normal
    # make orthonormal z axis, aligned with world down
    z = np.array([0.0, 0.0, -1.0])
    if np.abs(np.dot(z,Gy)) < 1e-6:
        # normal was pointing straight down.  reject this sample.
        #print('here')
        return np.inf, None

    Gz = z - np.dot(z,Gy)*Gy
    Gx = np.cross(Gy, Gz)
    R_WG = RotationMatrix(np.vstack((Gx, Gy, Gz)).T)
    p_GS_G = [0, 0.035, 0.11]

    # Try orientations from the center out
    min_pitch=-np.pi/3.0
    max_pitch=np.pi/3.0
    alpha = np.array([0.5, 0.65, 0.35, 0.8, 0.2, 1.0, 0.0])
    thetas = [0]
    for theta in thetas:#(min_pitch + (max_pitch - min_pitch)*alpha):
        # Rotate the object in the hand by a random rotation (around the normal).
        R_WG2 = R_WG.multiply(RotationMatrix.MakeYRotation(theta))

        # Use G for gripper frame.
        p_SG_W = - R_WG2.multiply(p_GS_G)
        p_WG = p_WS + p_SG_W 

        X_G = RigidTransform(R_WG2, p_WG)
        ik = PandaInverseKinematics(plant, plant_context, panda, avoid_names = avoid_names)
        p_WQ = X_G.translation()
        tol = np.ones(3)*0.01
        q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.]) 
        ik.AddPositionConstraint(p_WQ+tol, p_WQ+tol)
        ik.AddOrientationConstraint(X_G.rotation(), 0.01)
        ik.AddMinDistanceConstraint(0.01)
        prog = ik.get_prog()
        q = ik.get_q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q)
        prog.SetInitialGuess(q, q_nominal)
        result = Solve(prog)

        if not result.is_success():
            continue
        
        q = result.GetSolution(q)
        plant.SetPositions(plant_context, panda, q)
        plant.SetPositions(plant_context, plant.GetModelInstanceByName("hand"), [-0.04, 0.04])
        #print('evaluating cost')
        #print(X_G)
        cost = grasp_candidate_cost(X_G, plant_context, cloud, plant, scene_graph, scene_graph_context)
        #X_G = plant.GetFreeBodyPose(plant_context, body)
        if np.isfinite(cost):
            return cost, X_G

        #draw_grasp_candidate(q)

    return np.inf, None

def draw_grasp_candidate(q):
    builder = DiagramBuilder()
    
    builder = pydrake.systems.framework.DiagramBuilder()

    station = builder.AddSystem(PandaStation())
    station.SetupBinStation()

    plant = station.get_multibody_plant()
    panda = plant.GetModelInstanceByName("panda")

    station.Finalize()
    

    
    station_context = station.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(station_context)
    scene_graph = station.get_scene_graph()
    scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
    
    

    meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)
    
    diagram = builder.Build()
    plant.SetPositions(plant_context, panda, q)

    meshcat.load()
    diagram.Publish(context)
    
        

In [3]:
def make_environment_model(bin_name="bin0"):
    # Make one model of the environment, but the robot only gets to see the sensor outputs.
    
    directive = FindResource("models/two_bins_w_cameras.yaml")

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
    parser = Parser(plant)
    AddPackagePaths(parser)
    ProcessModelDirectives(LoadModelDirectives(directive), plant, parser)

    brick = parser.AddModelFromFile(FindResourceOrThrow("drake/examples/manipulation_station/models/061_foam_brick.sdf"), 'brick')
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('base_link', brick), RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0 +np.random.uniform(-0.1, 0.1)), [-.1+np.random.uniform(-0.1, 0.1), -.65, 0.09]))
    
    plant.Finalize()
    AddRgbdSensors(builder, plant, scene_graph)

    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment")

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)

    return diagram, context, plant, plant_context, scene_graph, scene_graph_context

def fix_collisions(plant, scene_graph, scene_graph_context):
    
    query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
    collision_pairs = query_object.ComputePointPairPenetration()
    
    for pair in collision_pairs:
        sA = GeometrySet([pair.id_A])
        sB = GeometrySet([pair.id_B])
        scene_graph.ExcludeCollisionsBetween(scene_graph_context, sA, sB)

In [ ]:
environment, environment_context, env_plant, env_plant_context, env_scene_graph, env_scene_graph_context = make_environment_model()

rng = np.random.default_rng()

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupBinStation()

plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")
c1 = AddShape(plant, Cylinder(0.03, 1), "c1")
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, -0.25, 0.5])) # add a model to test collisions
c2 = AddShape(plant, Cylinder(0.03, 1), "c2")
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c2", c2), RigidTransform([-0.3, 0, 0.5])) # add a model to test collisions


station.Finalize()

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


cloud = process_bin_point_cloud(environment, environment_context, ["camera0", "camera1", "camera2"], "bin0")

avoid_names= ['bin0', 'bin1', 'c1', 'c2']

costs = []
X_Gs = []
fix_collisions(plant, scene_graph, scene_graph_context)
for i in range(100):
    cost, X_G = generate_grasp_candidate_antipodal(plant_context, cloud, plant, scene_graph, scene_graph_context, panda, rng, avoid_names = avoid_names)#, meshcat=v.vis["sample"])
    print(cost)
    if np.isfinite(cost):      
        costs.append(cost)
        X_Gs.append(X_G)
        break

indices = np.asarray(costs).argsort()[:1]
print(X_Gs[indices[0]])

start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


trajectory = None


print("starting rrt")
rrt = PandaRRTompl(plant, scene_graph, plant_context, scene_graph_context, panda,
                        start_pose, 0, X_Gs[indices[0]], 10, avoid_names)
print("done rrt")

trajectory = rrt.get_trajectory()

q_traj_system = builder.AddSystem(TrajectorySource(trajectory))

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions

draw_open3d_point_cloud(meshcat.vis["cloud"], cloud, size=0.003)

builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("panda_position"))



diagram = builder.Build()

simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation

meshcat.start_recording()

simulator.AdvanceTo(10)

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6025...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7025/static/
Connected to meshcat-server.
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
